In [2]:
import pandas as pd
import chart_studio.plotly as py
import plotly.graph_objs as go 
from plotly.offline import init_notebook_mode,iplot,plot
init_notebook_mode(connected=True) 


In [3]:
df = pd.read_csv('covid-data.csv')
df.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,code
0,AFG,Asia,Afghanistan,24-02-2020,1.0,1.0,NaN,NaN,NaN,NaN,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498,AFG
1,AFG,Asia,Afghanistan,25-02-2020,1.0,0.0,NaN,NaN,NaN,NaN,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498,AFG
2,AFG,Asia,Afghanistan,26-02-2020,1.0,0.0,NaN,NaN,NaN,NaN,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498,AFG
3,AFG,Asia,Afghanistan,27-02-2020,1.0,0.0,NaN,NaN,NaN,NaN,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498,AFG
4,AFG,Asia,Afghanistan,28-02-2020,1.0,0.0,NaN,NaN,NaN,NaN,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498,AFG


In [4]:
tdf = df.groupby(['location','code']).sum()[['new_cases','new_deaths']]
adf = df.groupby('date').mean()[['total_cases','total_deaths']]

In [5]:
tdf = tdf.reset_index()

In [6]:
tdf['text'] = tdf['location']+'<br>'+'Total Cases ='+tdf['new_cases'].astype(str)+'<br>'+'Total Deaths ='+tdf['new_deaths'].astype(str)

In [7]:
tdf.head()

,location,code,new_cases,new_deaths,text
0,Afghanistan,AFG,53489.0,2277.0,Afghanistan<br>Total Cases =53489.0<br>Total D...
1,Albania,ALB,63595.0,1241.0,Albania<br>Total Cases =63595.0<br>Total Death...
2,Algeria,DZA,102144.0,2807.0,Algeria<br>Total Cases =102144.0<br>Total Deat...
3,Andorra,AND,8586.0,85.0,Andorra<br>Total Cases =8586.0<br>Total Deaths...
4,Angola,AGO,18193.0,416.0,Angola<br>Total Cases =18193.0<br>Total Deaths...


In [8]:
data = dict(type = 'choropleth',
        locations = tdf['code'],
        z = tdf['new_cases'],
        text = tdf['text'],
        colorbar = {'title' : 'Covid Cases'},
      ) 

In [9]:
layout = dict(title = 'Covid19 cases and deaths world wide',
    geo = dict(
        showframe = False,
        projection = {'type':'kavrayskiy7'}
    )
)

In [10]:
choromap = go.Figure(data = [data],layout = layout)
iplot(choromap)

In [11]:
adf.head()

,total_cases,total_deaths
date,,
01-01-2020,NaN,NaN
01-01-2021,879783.403141,20651.367232
01-02-2020,926.000000,259.000000
01-03-2020,2599.823529,460.923077
01-04-2020,10650.333333,774.584615


In [12]:
adf = df.groupby('date').mean()[['total_cases','total_deaths']]
adf =adf.reset_index()
adf['date'] = pd.to_datetime(adf['date'])
adf['month_numeric'] = adf['date'].apply(lambda x:x.month )

In [13]:
month_dict = {1:'January',2:'February',3:'March',4:'April',
        5:'May',6:'June',7:'July',8:'August',9:'September',10:'October',11:'November',12:'December'}
month_dict_reverse = {'January':1,'February':2,'March':3,'April':4,
        'May':5,'June':6,'July':7,'August':8,'September':9,'October':10,'November':11,'December':12}

In [14]:
adf['month'] = adf['month_numeric'].map(month_dict)

In [15]:
adf
new_df = adf.groupby('month').sum().reset_index().sort_values(by='month')
new_df['month_numeric']=new_df['month'].map(month_dict_reverse)
new_df = new_df.sort_values(by='month_numeric')

In [16]:
import numpy as np
import cufflinks as cf
cf.go_offline()
%matplotlib inline
# adf.iplot(kind='bar',y='total_cases',x='month',title='Global Data by months',xTitle='Months',yTitle='Cases in Millions')
new_df.iplot(kind='bar',y='total_cases',x='month',title='Global Covid Data by months',xTitle='Months',yTitle='Cases in Millions')
type(new_df['month'].iloc[0])

str

In [19]:
df2=df.pivot_table(values='new_cases',aggfunc='sum',columns='date',index='location')